In [1]:
import numpy as np
import scipy as sp 
import pandas as pd
import time
from scipy import stats as sps
from scipy import optimize
from scipy import special
from sklearn.linear_model import LogisticRegression
from IPython.display import display, Markdown

In [2]:
from datetime import datetime

## Example data from Chapter 2 of "What If?"

In [3]:
ch2data = {
    'L': [0]*8 + [1]*12,
    'A': [0]*4 + [1]*4 + [0]*3 + [1]*9,
    'Y': [0,1] + [0]*5 + [1]*3 + [0] + [1]*6 + [0]*3,
}
ch2names = ['Rheia', 'Kronos', 'Demeter', 'Hades', 'Hestia', 'Poseidon', 'Hera', 
            'Zeus', 'Artemis', 'Apollo', 'Leto', 'Ares', 'Athena', 'Hephaestus', 
            'Aphrodite', 'Cyclope', 'Persephone', 'Hermes', 'Hebe', 'Dionysus']
pd.DataFrame(ch2data, index=ch2names)

,L,A,Y
Rheia,0,0,0
Kronos,0,0,1
Demeter,0,0,0
Hades,0,0,0
Hestia,0,1,0
Poseidon,0,1,0
Hera,0,1,0
Zeus,0,1,1
Artemis,1,0,1
Apollo,1,0,1


## Basic function definitions

A utility function for generating all binary vectors of a given length.

In [4]:
def all_binary_vectors(length):
    result = [[]]
    while len(result[0]) < length:
        result = [ vector + [val] for vector in result for val in [False, True] ]
    return result

In [5]:
pd.DataFrame(all_binary_vectors(3))

,0,1,2
0,False,False,False
1,False,False,True
2,False,True,False
3,False,True,True
4,True,False,False
5,True,False,True
6,True,True,False
7,True,True,True


Calculate the standardized means for each level of A for a given dataset

In [6]:
def fit_phat_Y_given_L_A(L, A, Y):
    A = np.array(A)
    Y = np.array(Y)
    L = np.reshape(np.array(L), (len(Y), -1)) # force L to be a 2D array
    M = L.shape[1]
    A_levels = [False, True]
    L_levels = all_binary_vectors(M)
    phat_Y_given_L_A_table = np.empty((len(L_levels), len(A_levels)))
    
    for i,l in enumerate(L_levels):
        mask_l = (np.product(L == l, axis=1) != 0)
        for j,a in enumerate(A_levels):
            phat_Y_given_L_A_table[i,j] = np.mean(Y[mask_l * (A == a)])
    
    def phat_Y_given_L_A(L, A):
        A = np.array(A)
        L = np.reshape(np.array(L), (-1, M)) # force L to be a 2D array
        result = np.zeros(L.shape[0])
        for i,l in enumerate(L_levels):
            #print(L, l, )#np.product(L == l, axis=1) != 0)
            mask_l = (np.product(L == l, axis=1) != 0)
            for j,a in enumerate(A_levels):
                result[mask_l * (A == a)] = phat_Y_given_L_A_table[i,j]
        return result
    
    return phat_Y_given_L_A

In [7]:
def fit_phat_L(L):
    L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
    M = L.shape[1]
    L_levels = all_binary_vectors(M)
    phat_L_table = np.zeros((len(L_levels),))

    for i,l in enumerate(L_levels):
        mask_l = (np.product(L == l, axis=1) != 0)
        phat_L_table[i] = np.mean(mask_l)
        
    def phat_L(L):
        L = np.reshape(np.array(L), (-1,M)) # force L to be a 2D array
        result = np.zeros((L.shape[0],))
        for i,l in enumerate(L_levels):
            mask_l = (np.product(L == l, axis=1) != 0)
            result += mask_l * phat_L_table[i]
        return result
        
    return phat_L

In [8]:
def standardized_means_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
    L_levels = all_binary_vectors(L.shape[1])
    Y_mean = np.zeros((len(A_levels),))

    phat_L = fit_phat_L(L)
    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    
    for i,a in enumerate(A_levels):
        for l in L_levels:
            Y_mean[i] += (phat_Y_given_L_A(l, a) * phat_L(l))[0]

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [9]:
pd.DataFrame(standardized_means_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


In [10]:
def outcome_regression_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
    Y_mean = np.zeros((len(A_levels),))

    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    
    for i,a in enumerate(A_levels):
        Y_mean[i] += np.mean(phat_Y_given_L_A(L, a))

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [11]:
pd.DataFrame(outcome_regression_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


Calculate the inverse probability weighted means for each level of A for a given dataset

In [12]:
# Separately tabulate the freqency of A for each possible level of L
# (note: vectorized with respect to samples, so fast but a little ugly)
def fit_phat_A_given_L(L, A):
    A = np.array(A)
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A_levels = [False, True]
    L_levels = all_binary_vectors(L.shape[1])
    phat_A_given_L_table = np.zeros((len(L_levels), len(A_levels)))

    for i,l in enumerate(L_levels):
        mask_l = (np.product(L == l, axis=1) != 0)
        P_l = np.mean(mask_l)
        for j,a in enumerate(A_levels):
            phat_A_given_L_table[i,j] = np.mean(mask_l * (A == a))/P_l
            
    def phat_A_given_L(L):
        L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
        result = np.zeros((L.shape[0], len(A_levels)))
        for i,l in enumerate(L_levels):
            mask_l = (np.product(L == l, axis=1) != 0)
            for j,a in enumerate(A_levels):
                result[:,j] += mask_l * phat_A_given_L_table[i,j]
        return result
        
    return phat_A_given_L

In [13]:
def ip_weighted_means_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    phat_A_given_L = fit_phat_A_given_L(L, A)
    Y_mean = [np.mean(Y * (A == a) / phat_A_given_L(L)[:,i]) for i,a in enumerate(A_levels)]

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [14]:
pd.DataFrame(ip_weighted_means_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


Calculate the double-robust means for each level of A for a given dataset

In [15]:
def doubly_robust_means_for_A(L, A, Y):
    A = np.array(A)
    A_levels = [False, True]
    Y = np.array(Y)
    
    phat_A_given_L = fit_phat_A_given_L(L, A)
    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    f = phat_A_given_L(L)
    
    Y_mean = np.zeros((len(A_levels),))
    for i,a in enumerate(A_levels):
        q = phat_Y_given_L_A(L, a)
        Y_mean[i] = np.mean(q + (A == a) / f[:,i] * (Y - q))

    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [16]:
pd.DataFrame(doubly_robust_means_for_A(**ch2data))

,A_level,Y_mean
0,False,0.5
1,True,0.5


Calculate the means for each level of A using a marginal structural model (in this case, inverse probability of treatment weighting using logistic marginal models)

In [17]:
# Use a logistic marginal model for L
def fit_marginal_phat_A_given_L(L, A):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)
    
    model = LogisticRegression(solver='liblinear').fit(L, A)
    
    def phat_A_given_L(L):
        L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array
        return model.predict_proba(L)
        
    return phat_A_given_L

In [18]:
def fit_marginal_phat_P_Y_given_L_A(L, A, Y):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)

    marginal_phat_A_given_L = fit_marginal_phat_A_given_L(L, A)
    weights = A/marginal_phat_A_given_L(L)[:,1] + (1-A)/marginal_phat_A_given_L(L)[:,0]
    X = A[:,np.newaxis]
    model = LogisticRegression(solver='liblinear').fit(X, Y, weights)

    def phat_Y_given_L_A(L, A):
        A = np.array(A)
        L = np.reshape(np.array(L), (np.array(L).shape[0], -1)) # force L to be a 2D array        
        X = A[:,np.newaxis]
        return model.predict_proba(X)
        
    return phat_Y_given_L_A   

In [19]:
def marginal_structural_model_for_A(L, A, Y):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)
    Y = np.array(Y)

    marginal_phat_P_Y_given_L_A = fit_marginal_phat_P_Y_given_L_A(L, A, Y)
    
    A_levels = [False, True]
    Y_mean = [np.mean(marginal_phat_P_Y_given_L_A(L, np.ones_like(A)*a)[:,1]) for a in A_levels]
    
    return {
        'A_level': A_levels,
        'Y_mean': Y_mean,
    }

In [20]:
pd.DataFrame(marginal_structural_model_for_A(**ch2data))

,A_level,Y_mean
0,False,0.480048
1,True,0.510726


#### Calculate the log of the odds ratio using g-estimation

For a given link function $g(x)$, assume a linear effect of an intervention to create a structurally mean model (SMM):

$$g(\mathbb{E}(Y^a|L=l,A=a)) - g(\mathbb{E}(Y^{a=0}|L=l,A=a)) = \Psi_0 a + \Psi_L al$$

$$U = \mathbb{E}(Y^{a=0}|L=l,A=a) = g^{-1}(g(\mathbb{E}(Y^a|L=l,A=a)) - \Psi_0 a - \Psi_L al)$$


We first define a structural mean model (SMM) with link function  $g(x) = \mathrm{logit}(x)$ parameterized by $\Psi_0$ and $\Psi_L$:

$$\mathrm{logit}\left(\mathbb{E}\left(Y^a|L=l,A=a\right)\right) 
    - \mathrm{logit}\left(\mathbb{E}\left(Y^0|L=l,A=a\right)\right)
    = \gamma(a, l; \Psi_0, \Psi_L),$$
    
where

$$\gamma(a, l; \Psi_0, \Psi_L) = \Psi_0 a + \Psi_L al. $$
    
We can then construct a variable $U(\Psi_0, \Psi_L)$ such that

$$U(\Psi_0, \Psi_L) = \mathrm{expit}\left(\mathrm{logit}\left(\mathbb{E}\left(Y^a|L,A\right)\right) - \Psi_0 A - \Psi_L LA\right)$$

such that

$$\mathbb{E}\left(U\right | L,A)= \mathbb{E}\left(Y^{a=0} | L,A\right).$$

Because we assume $A \perp\!\!\perp Y^{a=0} | L$, we then find

$$\mathbb{E}\left(U | L,A \right)
    = \mathbb{E}\left(Y^{a=0} | L,A\right)
    = \mathbb{E}\left(Y^{a=0} | L\right)
    = \mathbb{E}\left(U | L \right)
.$$

More generally for an arbitrary function $f(A,L)$, $f(A,L) \perp\!\!\perp U | L$.  Thus 

$$0 = cov(f(A,L), U) | L
    = \sum_i (f(A_i, L_i) - \mathbb{E}(f(A,L_i) | L_i)) \cdot
        (U_i - \mathbb{E}(U | L_i))
.$$

We can then use this equality with the family of functions

$$f_i(A,L) = \frac{\partial U}{\partial \psi_i}$$

to provide a sufficient set of constraints to solve for $\psi$. 

Note: This function appears to work as long as the magnitude of the various effect sizes ($|\beta|$) is approximately $1$ or less, but usually generates erroneously large effect sizes if $|\beta|$ is greater than that. Maybe this is due to the vanishing derivatives of $\mathrm{expit}(x)$ function when $|x| >> 1$? 

In [70]:
def g_estimation_log_odds_for_A(L, A, Y):
    L = np.reshape(np.array(L), (len(A), -1)) # force L to be a 2D array
    A = np.array(A)
    a1 = np.ones_like(A)
    a0 = np.zeros_like(A)
    Y = np.array(Y)

    L_augmented = np.hstack([L, np.ones((L.shape[0],1))])
    phat_A_given_L = fit_phat_A_given_L(L, A)(L)[:,1]
    phat_Y_given_L_A = fit_phat_Y_given_L_A(L, A, Y)
    logit_phat_Y_given_L_A = special.logit(phat_Y_given_L_A(L,A))
    logit_phat_Y_given_L_a1 = special.logit(phat_Y_given_L_A(L,a1))
    logit_phat_Y_given_L_a0 = special.logit(phat_Y_given_L_A(L,a0))
    
    def gamma(L, A, psi):
        L_augmented_ = np.hstack([L, np.ones((L.shape[0],1))])
        return (L_augmented_ * A[:,np.newaxis]).dot(psi)
    
    def cov_dU_U(psi):
        U = special.expit(logit_phat_Y_given_L_A - gamma(L, A, psi))
        U_given_a0 = special.expit(logit_phat_Y_given_L_a0 - gamma(L, a0, psi))
        U_given_a1 = special.expit(logit_phat_Y_given_L_a1 - gamma(L, a1, psi))
        E_U_given_L = (phat_A_given_L * U_given_a1 + 
                       (1 - phat_A_given_L) * U_given_a0)
        
        dUdPsi = L_augmented * A[:,np.newaxis] * (U*(1-U))[:,np.newaxis]
        E_dUdPsi_given_a0 = (L_augmented * a0[:,np.newaxis] *
                               (U_given_a0*(1-U_given_a0))[:,np.newaxis])
        E_dUdPsi_given_a1 = (L_augmented * a1[:,np.newaxis] *
                               (U_given_a1*(1-U_given_a1))[:,np.newaxis])
        E_dUdPsi_given_L = (phat_A_given_L[:,np.newaxis] * E_dUdPsi_given_a1 + 
                       (1 - phat_A_given_L[:,np.newaxis]) * E_dUdPsi_given_a0)
        
        #print(psi)
        return np.sum((dUdPsi - E_dUdPsi_given_L) * (U - E_U_given_L)[:,np.newaxis], axis=0)
    opt_res = optimize.root(cov_dU_U, np.ones(L.shape[1] + 1)*1e-3)
    psi = opt_res.x
    print(opt_res)
    
    return np.mean(gamma(L, np.ones_like(A), psi))

In [71]:
g_estimation_log_odds_for_A(**ch2data)

    fjac: array([[-0.7244982 , -0.68927669],
       [ 0.68927669, -0.7244982 ]])
     fun: array([0., 0.])
 message: 'The solution converged.'
    nfev: 9
     qtf: array([ 6.41318184e-17, -1.31264618e-17])
       r: array([0.17617531, 0.18662121, 0.05873358])
  status: 1
 success: True
       x: array([-1.36215427e-16, -6.29422849e-17])


-1.44671541251651e-16

### Utilities

A simple utility function for summarizing results for a given model

In [72]:
def summarize_model(data, title='', description=''):
    start_time = time.monotonic()
    A = data['A']
    L = np.reshape(np.array(data['L']), (len(A), -1)) # force L to be a 2D array
    Y = data['Y']
    
    display(Markdown('### ' + title))
    display(Markdown(description))
    display(Markdown('#### Sample data'))
    sample_df = pd.concat([
        pd.DataFrame(L).add_prefix("L"), 
        pd.Series(A, name='A'), 
        pd.Series(Y, name='Y')
        ], axis=1)
    display(sample_df.iloc[:12])

    display(Markdown('#### Summary statistics'))
    stats = {
        '$\\bar L$': [np.mean(L[:,i]) for i in range(L.shape[1])],
        '$\\bar A$': np.mean(A),
        '$\\bar Y$': np.mean(Y),
    }
    if 'Y_a_0' in data:
        stats = stats | {
        '$\\bar Y^c_0$': np.mean(data['Y_a_0']),
        '$\\bar Y^c_1$': np.mean(data['Y_a_1']),
        }
    display(pd.DataFrame({'statistic': stats.values()}, index=stats.keys()))
    
    model_names = []
    Y_hat_a_0s = []
    Y_hat_a_1s = []
    beta_a_hats = []
    
    if 'Y_a_0' in data:
        model_names.append('Underlying counterfactual data')
        Y_hat_a_0s.append(np.mean(data['Y_a_0']))
        Y_hat_a_1s.append(np.mean(data['Y_a_1']))
        beta_a_hats.append(special.logit(np.mean(data['Y_a_1'])) -
            special.logit(np.mean(data['Y_a_0'])))

    models = [
        ["Standardized means", standardized_means_for_A],
        ["Outcome regression", outcome_regression_for_A], 
        ["Inverse probability weighted means (IPW)", ip_weighted_means_for_A],
        ["Doubly robust means", doubly_robust_means_for_A],
        ["Marginal structural model (MSM)", marginal_structural_model_for_A]
    ]
    for model_name, model in models:
        model_names.append(model_name)
        result = model(L, A, Y)
        Y_hat_a_0s.append(result['Y_mean'][0])
        Y_hat_a_1s.append(result['Y_mean'][1])
        beta_a_hats.append(special.logit(result['Y_mean'][1]) -
            special.logit(result['Y_mean'][0]))
        
    model_names.append('g-estimation')
    Y_hat_a_0s.append('')
    Y_hat_a_1s.append('')
    beta_a_hats.append(g_estimation_log_odds_for_A(L, A, Y))

        
    display(Markdown('#### Estimators'))
    display(pd.DataFrame({
        '$\\hat Y^{a=0}$':Y_hat_a_0s, 
        '$\\hat Y^{a=1}$':Y_hat_a_1s,
        '$\\hat \\beta_a$':beta_a_hats,
    }, index=model_names))
    
    total_time = time.monotonic() - start_time
    if total_time > 1:
        display(Markdown(f'*Total execution time*: {total_time} s'))

In [73]:
summarize_model(ch2data, 'Analysis of data from chapter 2', 'Validating the summary function with data from chapter 2')

### Analysis of data from chapter 2

Validating the summary function with data from chapter 2

#### Sample data

,L0,A,Y
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,1,0
5,0,1,0
6,0,1,0
7,0,1,1
8,1,0,1
9,1,0,1


#### Summary statistics

,statistic
$\bar L$,[0.6]
$\bar A$,0.65
$\bar Y$,0.5


    fjac: array([[-0.7244982 , -0.68927669],
       [ 0.68927669, -0.7244982 ]])
     fun: array([0., 0.])
 message: 'The solution converged.'
    nfev: 9
     qtf: array([ 6.41318184e-17, -1.31264618e-17])
       r: array([0.17617531, 0.18662121, 0.05873358])
  status: 1
 success: True
       x: array([-1.36215427e-16, -6.29422849e-17])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Standardized means,0.5,0.5,0.000000e+00
Outcome regression,0.5,0.5,0.000000e+00
Inverse probability weighted means (IPW),0.5,0.5,0.000000e+00
Doubly robust means,0.5,0.5,8.881784e-16
Marginal structural model (MSM),0.480048,0.510726,1.227615e-01
g-estimation,,,-1.446715e-16


## Testing with generated data

Start with a simple parameterized model...

In [74]:
def generate_simple_logistic_model_data(N=100000, P_L=0.5,
                                        beta_A_0=0, beta_A_L=0, 
                                        beta_Y_0=0, beta_Y_L=0, beta_Y_A=0, beta_Y_LA=0,
                                        debug_info=False):
    # reshape and broadcast values into appropriately sized arrays
    P_L = np.array(P_L).reshape((-1,)) # expand P_L to a 2d array if needed
    M = len(P_L)
    if np.isscalar(beta_A_L) == 1: # broadcast to appropriate shape if needed
        beta_A_L = np.array([beta_A_L]*M)
    else:
        beta_A_L = np.array([beta_A_L]).reshape((M,))
    if np.isscalar(beta_Y_L) == 1: # broadcast to appropriate shape if needed
        beta_Y_L = np.array([beta_Y_L]*M)
    else:
        beta_Y_L = np.array([beta_Y_L]).reshape((M,))
    if np.isscalar(beta_Y_LA) == 1: # broadcast to appropriate shape if needed
        beta_Y_LA = np.array([beta_Y_LA]*M)
    else:
        beta_Y_LA = np.array([beta_Y_LA]).reshape((M,))
    
    L = np.random.uniform(size=(N,)+np.array(P_L).shape) < P_L
    P_A_given_l = sps.logistic.cdf(beta_A_0 + L.dot(beta_A_L))
    A = np.random.uniform(size=N) < P_A_given_l
    P_Y_given_l_a_0 = sps.logistic.cdf(beta_Y_0 + L.dot(beta_Y_L) #+
                                     #A.dot(beta_Y_A) +
                                     #L.dot(beta_Y_LA)*A
                                     )
    P_Y_given_l_a_1 = sps.logistic.cdf(beta_Y_0 + L.dot(beta_Y_L) +
                                     beta_Y_A +#A.dot(beta_Y_A) +
                                     L.dot(beta_Y_LA)#L.dot(beta_Y_LA)*A
                                     )
    Y_a_0 = np.random.uniform(size=N) < P_Y_given_l_a_0
    Y_a_1 = np.random.uniform(size=N) < P_Y_given_l_a_1
    Y = Y_a_0 * (1 - A) + Y_a_1 * A
    
    return {
        'L' : L,
        'A' : A,
        'Y' : Y,
        'Y_a_0': Y_a_0,
        'Y_a_1': Y_a_1,
    } | ({
        'P_L' : P_L,
        'P_A_given_l' : P_A_given_l, 
        'P_Y_given_l_a' : P_Y_given_l_a,
    } if debug_info else {})

In [75]:
summarize_model(generate_simple_logistic_model_data(), 'Maximal entropy model', 'No interactions, 1:1 odds for L, A, and Y')

### Maximal entropy model

No interactions, 1:1 odds for L, A, and Y

#### Sample data

,L0,A,Y
0,False,True,0
1,False,True,0
2,False,False,0
3,True,False,0
4,False,False,1
5,True,True,1
6,True,False,1
7,True,False,0
8,False,True,1
9,True,True,0


#### Summary statistics

,statistic
$\bar L$,[0.49849]
$\bar A$,0.49923
$\bar Y$,0.49889
$\bar Y^c_0$,0.49939
$\bar Y^c_1$,0.49911


    fjac: array([[-0.70711366, -0.7070999 ],
       [ 0.7070999 , -0.70711366]])
     fun: array([6.91772851e-13, 3.43781319e-13])
 message: 'The solution converged.'
    nfev: 6
     qtf: array([-1.41974946e-08,  1.06114953e-09])
       r: array([1101.48634121, 1655.57904499,  554.09534739])
  status: 1
 success: True
       x: array([-0.00788578,  0.00677938])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.49939,0.49911,-0.001120
Standardized means,0.498534,0.499246,0.002848
Outcome regression,0.498534,0.499246,0.002848
Inverse probability weighted means (IPW),0.498534,0.499246,0.002848
Doubly robust means,0.498534,0.499246,0.002848
Marginal structural model (MSM),0.498534,0.499246,0.002848
g-estimation,,,0.002848


In [76]:
summarize_model(generate_simple_logistic_model_data(beta_Y_A=-1),
                'Simple randomized protective intervention', 'Assumes no effect from covariate')

### Simple randomized protective intervention

Assumes no effect from covariate

#### Sample data

,L0,A,Y
0,True,True,0
1,False,False,1
2,True,False,1
3,True,False,0
4,True,True,0
5,True,False,0
6,True,False,0
7,True,True,0
8,False,False,0
9,True,True,1


#### Summary statistics

,statistic
$\bar L$,[0.50105]
$\bar A$,0.50036
$\bar Y$,0.38349
$\bar Y^c_0$,0.49697
$\bar Y^c_1$,0.27013


    fjac: array([[-0.74204632, -0.67034861],
       [ 0.67034861, -0.74204632]])
     fun: array([8.22330389e-10, 1.62207279e-09])
 message: 'The solution converged.'
    nfev: 13
     qtf: array([6.28779419e-06, 2.41161774e-06])
       r: array([ 959.45919264, 1631.69108752,  635.98555392])
  status: 1
 success: True
       x: array([ 0.01349245, -0.98484527])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.49697,0.27013,-0.981843
Standardized means,0.496582,0.270565,-0.978085
Outcome regression,0.496582,0.270565,-0.978085
Inverse probability weighted means (IPW),0.496582,0.270565,-0.978085
Doubly robust means,0.496582,0.270565,-0.978085
Marginal structural model (MSM),0.496572,0.270575,-0.977997
g-estimation,,,-0.978085


*Total execution time*: 1.4838208170840517 s

In [77]:
summarize_model(generate_simple_logistic_model_data(beta_Y_A=-1, beta_Y_L=1),
                'Randomized, covariate is risk, intervention is protective',
                '1:1 odds of covariate')

### Randomized, covariate is risk, intervention is protective

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,True,True,0
1,True,True,1
2,True,False,1
3,False,False,1
4,False,True,0
5,True,False,1
6,False,False,0
7,True,False,1
8,False,False,1
9,False,True,0


#### Summary statistics

,statistic
$\bar L$,[0.49931]
$\bar A$,0.50293
$\bar Y$,0.50084
$\bar Y^c_0$,0.61546
$\bar Y^c_1$,0.38564


    fjac: array([[-0.70998891, -0.70421286],
       [ 0.70421286, -0.70998891]])
     fun: array([-2.70196253e-13, -6.17616670e-13])
 message: 'The solution converged.'
    nfev: 15
     qtf: array([-1.77152592e-09,  1.49253371e-10])
       r: array([ 703.69039516, 1586.73390688,  527.52315631])
  status: 1
 success: True
       x: array([-0.02669723, -0.9853869 ])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.61546,0.38564,-0.935999
Standardized means,0.616807,0.386232,-0.939196
Outcome regression,0.616807,0.386232,-0.939196
Inverse probability weighted means (IPW),0.616807,0.386232,-0.939196
Doubly robust means,0.616807,0.386232,-0.939196
Marginal structural model (MSM),0.616791,0.386243,-0.939079
g-estimation,,,-0.998717


*Total execution time*: 1.3497557069640607 s

In [78]:
summarize_model(generate_simple_logistic_model_data(beta_A_0=-2, beta_Y_A=-1, beta_Y_L=1),
                'Randomized, covariate is risk, rare intervention is protective',
                '1:1 odds of covariate')

### Randomized, covariate is risk, rare intervention is protective

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,True,False,1
1,False,False,1
2,True,False,0
3,False,False,0
4,True,True,0
5,True,False,1
6,False,False,1
7,True,False,1
8,True,False,0
9,False,True,1


#### Summary statistics

,statistic
$\bar L$,[0.49642]
$\bar A$,0.11759
$\bar Y$,0.58959
$\bar Y^c_0$,0.61582
$\bar Y^c_1$,0.38364


    fjac: array([[-0.70926174, -0.70494523],
       [ 0.70494523, -0.70926174]])
     fun: array([-2.29229158e-13, -2.29229158e-13])
 message: 'The solution converged.'
    nfev: 15
     qtf: array([-1.02365776e-09,  7.50120608e-11])
       r: array([298.60961732, 657.51488713, 217.08080348])
  status: 1
 success: True
       x: array([ 0.05630695, -1.00580206])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.61582,0.38364,-0.945970
Standardized means,0.616089,0.39057,-0.917896
Outcome regression,0.616089,0.39057,-0.917896
Inverse probability weighted means (IPW),0.616089,0.39057,-0.917896
Doubly robust means,0.616089,0.39057,-0.917896
Marginal structural model (MSM),0.616069,0.390607,-0.917661
g-estimation,,,-0.977850


*Total execution time*: 1.227130011888221 s

In [46]:
summarize_model(generate_simple_logistic_model_data(P_L=0.75, beta_A_0=-2, beta_Y_A=-1, beta_Y_L=1),
                'Randomized, common covariate is risk, rare intervention is protective',
                '1:1 odds of covariate')

### Randomized, common covariate is risk, rare intervention is protective

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,True,False,0
1,True,False,1
2,True,False,1
3,True,False,0
4,True,False,1
5,True,False,1
6,True,False,1
7,True,False,1
8,False,False,1
9,True,False,0


#### Summary statistics

,statistic
$\bar L$,[0.74776]
$\bar A$,0.11835
$\bar Y$,0.64809
$\bar Y^c_0$,0.67546
$\bar Y^c_1$,0.44239


[1.69177211e-13 9.55844272e-14]


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.67546,0.44239,-0.964453
Standardized means,0.674929,0.448144,-0.938734
Outcome regression,0.674929,0.448144,-0.938734
Inverse probability weighted means (IPW),0.674929,0.448144,-0.938734
Doubly robust means,0.674929,0.448144,-0.938734
Marginal structural model (MSM),0.674907,0.448189,-0.938455
g-estimation,,,-0.981503


*Total execution time*: 1.3636892520007677 s

In [85]:
summarize_model(generate_simple_logistic_model_data(P_L=0.75,
                                                    beta_A_0=-2, beta_A_L=1, 
                                                    beta_Y_A=-1, beta_Y_L=1),
                'Common covariate is risk for disease and intervention, intervention is protective',
                '')

### Common covariate is risk for disease and intervention, intervention is protective

#### Sample data

,L0,A,Y
0,True,False,1
1,True,True,0
2,True,False,1
3,True,False,1
4,True,True,0
5,False,False,1
6,True,False,0
7,True,False,1
8,True,True,1
9,True,False,0


#### Summary statistics

,statistic
$\bar L$,[0.74887]
$\bar A$,0.23044
$\bar Y$,0.61784
$\bar Y^c_0$,0.67338
$\bar Y^c_1$,0.44002


    fjac: array([[-0.70901946, -0.70518892],
       [ 0.70518892, -0.70901946]])
     fun: array([6.98829987e-11, 6.59351684e-11])
 message: 'The solution converged.'
    nfev: 14
     qtf: array([-1.83565844e-07,  4.76473023e-09])
       r: array([ 886.73656506, 1436.59169709,  105.46258351])
  status: 1
 success: True
       x: array([ 0.01653767, -1.04397381])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.67338,0.44002,-0.964593
Standardized means,0.672712,0.434421,-0.984314
Outcome regression,0.672712,0.434421,-0.984314
Inverse probability weighted means (IPW),0.672712,0.434421,-0.984314
Doubly robust means,0.672712,0.434421,-0.984314
Marginal structural model (MSM),0.672688,0.434477,-0.983978
g-estimation,,,-1.031589


*Total execution time*: 1.212496135965921 s

In [48]:
summarize_model(generate_simple_logistic_model_data(P_L=0.5,
                                                    beta_A_0=0, 
                                                    beta_Y_A=1, beta_Y_L=1, beta_Y_LA=-2),
                'Covariate and intervention are risks, but negate eachother',
                '1:1 odds of covariate')

### Covariate and intervention are risks, but negate eachother

1:1 odds of covariate

#### Sample data

,L0,A,Y
0,False,True,1
1,False,False,1
2,True,True,1
3,False,True,1
4,False,False,0
5,True,False,1
6,True,True,0
7,True,True,0
8,False,True,0
9,False,True,1


#### Summary statistics

,statistic
$\bar L$,[0.50106]
$\bar A$,0.49791
$\bar Y$,0.61386
$\bar Y^c_0$,0.61616
$\bar Y^c_1$,0.61367


[1.32859186e-11 1.17037437e-12]


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.61616,0.61367,-0.010515
Standardized means,0.615228,0.613503,-0.007281
Outcome regression,0.615228,0.613503,-0.007281
Inverse probability weighted means (IPW),0.615228,0.613503,-0.007281
Doubly robust means,0.615228,0.613503,-0.007281
Marginal structural model (MSM),0.615223,0.613503,-0.007261
g-estimation,,,-0.005523


*Total execution time*: 1.3164625389617868 s

## Simulations with multi-dimensional $L$

In [49]:
summarize_model(generate_simple_logistic_model_data(P_L=(0.5,0.5)), 
    'Multidimensional maximal entropy model', 'No interactions, 1:1 odds for L, A, and Y')

### Multidimensional maximal entropy model

No interactions, 1:1 odds for L, A, and Y

#### Sample data

,L0,L1,A,Y
0,True,True,False,0
1,False,True,True,0
2,True,False,False,1
3,False,False,True,1
4,False,True,False,1
5,False,False,False,1
6,True,True,False,1
7,False,True,False,1
8,True,True,False,1
9,False,False,False,1


#### Summary statistics

,statistic
$\bar L$,"[0.49963, 0.50088]"
$\bar A$,0.50311
$\bar Y$,0.50223
$\bar Y^c_0$,0.50164
$\bar Y^c_1$,0.50079


[-2.19654328e-11 -1.28890489e-11 -2.15323061e-11]


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.50164,0.50079,-0.003400
Standardized means,0.501064,0.503367,0.009215
Outcome regression,0.501064,0.503367,0.009215
Inverse probability weighted means (IPW),0.501064,0.503367,0.009215
Doubly robust means,0.501064,0.503367,0.009215
Marginal structural model (MSM),0.50107,0.50337,0.009200
g-estimation,,,0.009213


*Total execution time*: 1.2847531989682466 s

In [80]:
summarize_model(generate_simple_logistic_model_data(
    P_L=(0.2,0.5), beta_A_0=-2, beta_A_L=(4,1), beta_Y_L=(2, 1), beta_Y_LA=(-2,2)), 
    'Treatment already targeted',
    'Treatment only benefits rare condition, commonly causes harm. Common risk factor present.')

### Treatment already targeted

Treatment only benefits rare condition, commonly causes harm. Common risk factor present.

#### Sample data

,L0,L1,A,Y
0,False,False,False,0
1,True,False,True,0
2,False,False,False,0
3,False,True,False,1
4,False,False,False,0
5,False,False,False,0
6,False,False,True,0
7,True,True,True,1
8,False,False,True,0
9,True,False,True,1


#### Summary statistics

,statistic
$\bar L$,"[0.20042, 0.49929]"
$\bar A$,0.33986
$\bar Y$,0.66574
$\bar Y^c_0$,0.67483
$\bar Y^c_1$,0.72706


    fjac: array([[-7.07106781e-01,  5.55111512e-17, -7.07106781e-01],
       [ 3.51125129e-01, -8.67999014e-01, -3.51125129e-01],
       [-6.13767989e-01, -4.96565919e-01,  6.13767989e-01]])
     fun: array([0., 0., 0.])
 message: 'The solution converged.'
    nfev: 68
     qtf: array([ 1.19180456e-13,  4.15782606e-14, -7.26789491e-14])
       r: array([ 1.72462106e+01, -5.50204861e-13,  2.05793084e+02, -1.05077291e-12,
        9.36259496e+01, -1.63658497e+02])
  status: 1
 success: True
       x: array([ -1.91281123, -34.07047049,  -0.04394321])


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.67483,0.72706,0.249644
Standardized means,0.675612,0.721826,0.219861
Outcome regression,0.675612,0.721826,0.219861
Inverse probability weighted means (IPW),0.675612,0.721826,0.219861
Doubly robust means,0.675612,0.721826,0.219861
Marginal structural model (MSM),0.675652,0.72204,0.220743
g-estimation,,,-17.438354


*Total execution time*: 5.738089210004546 s

In [51]:
summarize_model(generate_simple_logistic_model_data(
    P_L=(0.5,0.2), beta_A_0=-2, beta_A_L=(0,4), beta_Y_L=(1, 2), beta_Y_LA=(2,-2)), 
    'Treatment already targeted, swapped L values',
    'Treatment only benefits rare condition, commonly causes harm. Common risk factor present.')

### Treatment already targeted, swapped L values

Treatment only benefits rare condition, commonly causes harm. Common risk factor present.

#### Sample data

,L0,L1,A,Y
0,False,True,True,0
1,False,False,False,0
2,False,False,False,0
3,False,False,True,0
4,True,False,False,0
5,True,True,True,1
6,False,False,False,1
7,False,True,False,1
8,False,False,False,0
9,True,False,False,0


#### Summary statistics

,statistic
$\bar L$,"[0.50294, 0.19868]"
$\bar A$,0.27076
$\bar Y$,0.65167
$\bar Y^c_0$,0.67356
$\bar Y^c_1$,0.7278


[0. 0. 0.]


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.67356,0.7278,0.259158
Standardized means,0.671612,0.731849,0.288540
Outcome regression,0.671612,0.731849,0.288540
Inverse probability weighted means (IPW),0.671612,0.731849,0.288540
Doubly robust means,0.671612,0.731849,0.288540
Marginal structural model (MSM),0.671839,0.733821,0.297580
g-estimation,,,-17.898331


*Total execution time*: 5.660497158009093 s

In [36]:
summarize_model(generate_simple_logistic_model_data(
    P_L=(0.2,0.5), beta_A_0=-2/4, beta_A_L=(4/4,0/4), beta_Y_L=(2/4, 1/4), beta_Y_LA=(-2/4,2/4)), 
    'Treatment already targeted, effect size reduced',
    'Treatment only benefits rare condition, commonly causes harm. Common risk factor present.')

### Treatment already targeted, effect size reduced

Treatment only benefits rare condition, commonly causes harm. Common risk factor present.

#### Sample data

,L0,L1,A,Y
0,False,False,True,1
1,False,False,False,0
2,False,True,True,0
3,False,False,False,1
4,False,True,False,1
5,False,True,True,1
6,True,False,True,0
7,False,True,False,0
8,False,False,False,1
9,False,False,False,0


#### Summary statistics

,statistic
$\bar L$,"[0.1993, 0.49921]"
$\bar A$,0.42551
$\bar Y$,0.56642
$\bar Y^c_0$,0.55424
$\bar Y^c_1$,0.58935


#### Estimators

,$\hat Y^{a=0}$,$\hat Y^{a=1}$,$\hat \beta_a$
Underlying counterfactual data,0.55424,0.58935,0.143462
Standardized means,0.556017,0.591894,0.146791
Outcome regression,0.556017,0.591894,0.146791
Inverse probability weighted means (IPW),0.556017,0.591894,0.146791
Doubly robust means,0.556017,0.591894,0.146791
Marginal structural model (MSM),0.556032,0.591944,0.146932
g-estimation,,,0.155936


*Total execution time*: 1.7588757060002536 s

## Notes for next time

- Consider trying g-estimation on a generated dataset with a continuous (vs binary) outcome with an exponential or linear link function 
- Consider looking a propensity weighting of general hospital population and population with CAM screens to use model for delirium prediction with patients with CAM screens to predict risk in general population (a censoring problem)
- Consider simulating L to have same range as actual data (or sampled from actual data?) or subset of actual L?  Likely better to wait until new data import pipeline is working.
- (Later) will need to think about feature selection for real model; may want to filter first for features correlated with outcome or intervention, then go through by hand for likely causal relation given domain knowledge.
- Consider replacing existing non-parametric estimators - likely will require trying different models with outcome regression, IPW, and doubly robust (all should match if models are working properly).
- Could consider using using difference between IPW and outcome regression as part of loss function? (unproven - only guarantees that they match)